In [1]:
import requests
import copy
from bs4 import BeautifulSoup
from collections import defaultdict
HEADER = {
    'Accept':'*/*',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'en-GB,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6',
    'Connection':'keep-alive',
    'Referer':'https://dictionary.cambridge.org/dictionary/english-chinese-simplified/',
    'Host':'dictionary.cambridge.org',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
}

In [2]:
examples=[]
case = {"label":str(),"exp_e":str(),"exp_c":str(),"examples":[]}
phrase = {"form":str(),"phrase_info":str(),"exp_e":str(),"exp_c":str(),"examples":[]}

dense = {"dense_info":str(), "cases":[], "phrases":[]}
classs = {"class":str(),"pronounciation":str(),"denses":[]}
single = {"word":str(),"classes":[]}

In [3]:
url = 'https://dictionary.cambridge.org/dictionary/english-chinese-simplified/stall'
response = requests.get(url, headers=HEADER)

In [4]:
response.encoding = 'utf-8'

In [5]:
res_parsed = BeautifulSoup(response.text, 'html.parser')

In [6]:
single = {"word":str(),"classes":[]}
single["word"] = res_parsed.find("span", {"class": "hw dhw"}).text
for each_class in res_parsed.find_all("div", {"class": "pr entry-body__el"}): # For a class
    classs = {"class":str(),"pronounciation":str(),"denses":[]}
    classs["class"] = each_class.find("span", {"class": "pos dpos"}).get_text()
    classs["pronounciation"] = each_class.find("span", {"class": "pron dpron"}).get_text()
    
    for each_dense in each_class.find_all("div", {"class": "pr dsense"}): # For a dense
        dense = {"dense_info":str(), "cases":[], "phrases":[]}
        dense["dense_info"] = each_dense.find("h3", {"class": "dsense_h"}).get_text()
        
        main_block = each_dense.find('div', {'class': 'sense-body dsense_b'})
        for each_case in main_block.findChildren("div", {"data-wl-senseid": True} , recursive=False):
            case = {"label":str(),"exp_e":str(),"exp_c":str(),"examples":[]}
            case["label"]=each_case.find("a", {"href":True}).get_text().replace(" ", "")
            case["exp_e"]=each_case.find("div", {"class":"def ddef_d db"}).get_text()
            second_block = each_case.find("div",{"class":"def-body ddef_b"})
            case["exp_c"]= second_block.findChildren("span", {"lang": "zh-Hans"} , recursive=False)[0].get_text()
            examples=[]
            for each_example in second_block.findChildren("div", {"class": "examp dexamp"} , recursive=False):
                examples.append({"exa_e":each_example.find("span",{"class":"eg deg"}).get_text(),
                                 "exa_c":each_example.find("span",{"lang":"zh-Hans"}).get_text()})
            case["examples"] = copy.deepcopy(examples)
            dense["cases"].append(case)

        for each_phrase in main_block.findChildren("div", {"class": "pr phrase-block dphrase-block lmb-25"} , recursive=False)+main_block.findChildren("div", {"class": "pr phrase-block dphrase-block"} , recursive=False):
            phrase = {"form":str(),"phrase_info":str(),"exp_e":str(),"exp_c":str(),"examples":[]}
            phrase["form"] = each_phrase.find("span",{"class":"phrase-title dphrase-title"}).get_text()
            phrase["phrase_info"] = each_phrase.find("span",{"class":"phrase-info dphrase-info"}).get_text() if each_phrase.find("span",{"class":"phrase-info dphrase-info"}) else None
            second_block = each_phrase.findChildren("div", {"data-wl-senseid": True} , recursive=True)[0]
            phrase["exp_e"] = second_block.find("div", {"class":"def ddef_d db"}).get_text()
            phrase["exp_c"] = second_block.find("span", {"lang": "zh-Hans"}).get_text()
            examples = []
            for each_example in second_block.find_all("div", {"class": "examp dexamp"}):
                examples.append({"exa_e":each_example.find("span",{"class":"eg deg"}).get_text(),
                                 "exa_c":each_example.find("span",{"lang":"zh-Hans"}).get_text()})
            phrase["examples"] = copy.deepcopy(examples)
            # print(examples)
            # print(phrase,"\n")
            dense["phrases"].append(phrase)
        # print(dense,"\n")
        classs["denses"].append(dense)
    single["classes"].append(classs)

In [7]:
single

{'word': 'stall',
 'classes': [{'class': 'noun',
   'pronounciation': '/stɔːl/',
   'denses': [{'dense_info': 'stall noun \n            (SHOP)\n      ',
     'cases': [{'label': '[C]',
       'exp_e': 'a large table or a small shop with an open front from which goods are sold in a public place',
       'exp_c': '货摊，摊位；售货亭',
       'examples': [{'exa_e': 'In the village market, the stalls are piled high with local vegetables.',
         'exa_c': '村子集市的摊位上高高地堆着当地产的蔬菜。'}]}],
     'phrases': []},
    {'dense_info': 'stall noun \n            (SMALL AREA)\n      ',
     'cases': [{'label': '[C]',
       'exp_e': 'a small closed area within a farm building in which there is space for one animal to be kept',
       'exp_c': '（农场建筑中圈养单只牲畜的）隔栏，分隔间',
       'examples': []},
      {'label': '[C]',
       'exp_e': 'a small area of a room that is separated from the main part of the room by walls or curtains',
       'exp_c': '（房间内的）小隔间',
       'examples': [{'exa_e': 'There was one bathroom with a s

In [30]:
output = str()

In [31]:
output += "{{c1::"+single["word"]+"}}<br>"

In [32]:
for each_class in single["classes"]:
    output += f"""<span style="font-size:20px;font-weight:bold;">{each_class["class"]}</span>&nbsp"""
    output += "{{c1::"+each_class["pronounciation"]+"}}<br>"
    for each_dense in each_class["denses"]:
        output += "<hr color=#7851a9 size=3;>"
        # output += f"""<span color=#7851a9>{each_dense["dense_info"]}</span><br>"""
        for each_case in each_dense["cases"]:
            output += "<hr color=#FF8C00 size=1;>"
            output += f"""<span style="font-size:12px;color:#FF8C00;">{each_case["label"]}</span><br>"""
            output += f"""<span style="font-weight:bold;color:#00308F;">{each_case["exp_e"]}</span><br>"""
            output += f"""<span style="font-weight:bold;color:#007FFF;font-family:KaiTi;">{each_case["exp_c"]}</span><br>"""
            
            if len(each_case["examples"])>0:
                output += "<ul>"
                for each_example in each_case["examples"]:
                    output += "<li>"
                    output += f"""<span style="color:#00308F;font-style:italic;">{each_example["exa_e"]}</span><br>"""
                    output += f"""<span style="color:#007FFF;font-style:italic;font-family:KaiTi;">{each_example["exa_c"]}</span><br>"""
                    output += "</li>"
                output += "</ul>"
        for each_phrase in each_dense["phrases"]:
            output += "<hr color=#FF8C00 size=1;>"
            output += """<div style="background-color:#F0F8FF;padding:5px;">"""
            
            output += f"""<span style="color:#FF8C00;">{each_phrase["form"]} {each_phrase["phrase_info"]}</span><br>"""
            output += f"""<span style="font-weight:bold;color:#00308F;">{each_phrase["exp_e"]}</span><br>"""
            output += f"""<span style="font-weight:bold;color:#007FFF;font-family:KaiTi;">{each_phrase["exp_c"]}</span><br>"""
            
            if len(each_phrase["examples"])>0:
                output += "<ul>"
                for each_example in each_phrase["examples"]:
                    output += "<li>"
                    output += f"""<span style="color:#00308F;font-style:italic;">{each_example["exa_e"]}</span><br>"""
                    output += f"""<span style="color:#007FFF;font-style:italic;font-family:KaiTi;">{each_example["exa_c"]}</span><br>"""
                    output += "</li>"
                output += "</ul>"

            output += """</div>"""

In [33]:
print(output)

{{c1::stall}}<br><span style="font-size:20px;font-weight:bold;">noun</span>&nbsp{{c1::/stɔːl/}}<br><hr color=#7851a9 size=3;><hr color=#FF8C00 size=1;><span style="font-size:12px;color:#FF8C00;">[C]</span><br><span style="font-weight:bold;color:#00308F;">a large table or a small shop with an open front from which goods are sold in a public place</span><br><span style="font-weight:bold;color:#007FFF;font-family:KaiTi;">货摊，摊位；售货亭</span><br><ul><li><span style="color:#00308F;font-style:italic;">In the village market, the stalls are piled high with local vegetables.</span><br><span style="color:#007FFF;font-style:italic;font-family:KaiTi;">村子集市的摊位上高高地堆着当地产的蔬菜。</span><br></li></ul><hr color=#7851a9 size=3;><hr color=#FF8C00 size=1;><span style="font-size:12px;color:#FF8C00;">[C]</span><br><span style="font-weight:bold;color:#00308F;">a small closed area within a farm building in which there is space for one animal to be kept</span><br><span style="font-weight:bold;color:#007FFF;font-family:

In [25]:
single["word"] = res_parsed.find("span", {"class": "hw dhw"}).text

In [29]:
for cla in res_parsed.find_all("div", {"class": "pr entry-body__el"}):
    print(cla)

<div class="pr entry-body__el"><div class="cid" id="caldcns-1"></div><div class="pos-header dpos-h"><div class="di-title"><span class="headword hdb tw-bw dhw dpos-h_hw"><span class="hw dhw">tell</span></span></div><div class="posgram dpos-g hdib lmr-5"><span class="pos dpos" title="A word that describes an action, condition or experience.">verb</span></div> <div></div><span class="uk dpron-i"><span class="region dreg">uk</span><span class="daud">
<audio class="hdn" controlslist="nodownload" id="audio1" preload="none">
<div class="hdib" fallback="">
<p>Your browser doesn't support HTML5 audio</p>
</div>
<source src="/media/english-chinese-simplified/uk_pron/u/ukt/uktel/uktelex002.mp3" type="audio/mpeg"/>
<source src="/media/english-chinese-simplified/uk_pron_ogg/u/ukt/uktel/uktelex002.ogg" type="audio/ogg"/>
</audio>
<div class="i i-volume-up c_aud htc hdib hp hv-1 fon tcu tc-bd lmr-10 lpt-3" onclick="audio1.load(); audio1.play();" role="button" tabindex="0" title="Listen to the British

In [30]:
classes = res_parsed.find_all("div", {"class": "pr entry-body__el"})[0]

In [31]:
classes.find("span", {"class": "pos dpos"}).get_text()

'verb'

In [32]:
classes.find("span", {"class": "pron dpron"}).get_text()

'/tel/'

In [35]:
temp_dense = classes.find_all("div", {"class": "pr dsense"})[1]

In [36]:
temp_dense.find("h3", {"class": "dsense_h"}).text

'tell verb \n            (KNOW)\n      '

In [37]:
mid = temp_dense.find('div', {'class': 'sense-body dsense_b'})
temp_cases = mid.findChildren("div", {"data-wl-senseid": True} , recursive=False)
temp_phrases = mid.findChildren("div", {"class": "pr phrase-block dphrase-block lmb-25"} , recursive=False)+mid.findChildren("div", {"class": "pr phrase-block dphrase-block"} , recursive=False)

In [38]:
temp_cases[1].find("a", {"href":True}).get_text().replace(" ", "")

'[T]'

In [39]:
temp_cases[1].find("div", {"class":"def ddef_d db"}).get_text()

'If something tells you something, it gives you information.'

In [42]:
mid = temp_cases[1].find("div",{"class":"def-body ddef_b"})

In [43]:
mid.findChildren("span", {"lang": "zh-Hans"} , recursive=False)[0].get_text()

'（事物）显露，显示'

In [44]:
temp_examples = mid.findChildren("div", {"class": "examp dexamp"} , recursive=False)

In [45]:
temp_examples[0].find("span",{"class":"eg deg"}).get_text()

'What does the survey tell us about the lives of teenagers?'

In [46]:
temp_examples[0].find("span",{"lang":"zh-Hans"}).get_text()

'这次调查给我们提供了有关青少年生活的什么信息？'

In [47]:
temp_phrases[-1].find("span",{"class":"phrase-title dphrase-title"}).get_text()

'you never can tell'

In [48]:
temp_phrases[-1].find("span",{"class":"var dvar"}).get_text()

'(also you can never tell)'

In [50]:
temp_phrases_cases = temp_phrases[-1].findChildren("div", {"data-wl-senseid": True} , recursive=True)

In [51]:
temp_phrases_cases[0].find("div", {"class":"def ddef_d db"}).get_text()

'said to mean that you can never know or be certain'

In [64]:
# mid = temp_phrases_cases[0].find("div",{"class":"def-body ddef_b"})

In [65]:
temp_phrases_cases[0].find("span", {"lang": "zh-Hans"}).get_text()

'很难说，没法预料'

In [63]:
# mid.findChildren("span", {"lang": "zh-Hans"} , recursive=False)[0].get_text()

In [86]:
temp_phrases_examples = mid.findChildren("div", {"class": "examp dexamp"} , recursive=False)

In [87]:
temp_phrases_examples[0].find("span",{"class":"eg deg"}).get_text()

'Who knows what will happen to Peter and me in the future - you can never tell.'

In [88]:
temp_phrases_examples[0].find("span",{"lang":"zh-Hans"}).get_text()

'谁知道彼得和我将来会怎么样呢——谁也无法预料。'

In [21]:
res_parsed

<!DOCTYPE html>

<html lang="en">
<head>
<title>stall in Simplified Chinese - Cambridge Dictionary</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="stall translate: 商店, 货摊，摊位；售货亭, 小面积, （农场建筑中圈养单只牲畜的）隔栏，分隔间, （房间内的）小隔间, 座位, 发动机, （使）（发动机）熄火, 推迟, 拖延, 拖住（某人）, 停止进展…. Learn more in the Cambridge English-Chinese simplified Dictionary." name="description"/>
<meta content="stall, chinese (simplified), dictionary, english, british, british english, definition, define, meaning, spelling, conjugation, audio pronunciation, free, online" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
<script charset="UTF-8" src="https://cdn.cookielaw.org/consent/tcf.stub.js" type="text/javascript"></script>
<script charset="UTF-8" type="text/javascript">
                window.isGDPRMode = function() {
                    return true;
       